In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import datetime

In [ ]:
aust_yld_crv =  [
                (datetime.date(2022,9,20),-0.73),
                (datetime.date(2023,7,15),-0.745),
                (datetime.date(2024,7,15),-0.731),
                (datetime.date(2025,10,20),-0.715),
                (datetime.date(2026,10,20),-0.635),
                (datetime.date(2027,4,20),-0.600),
                (datetime.date(2028,2,20),-0.516),
                (datetime.date(2029,2,20),-0.403),
                (datetime.date(2030,2,20),-0.319),
                (datetime.date(2031,2,20),-0.234),
                (datetime.date(2037,3,15),0.026),
                (datetime.date(2040,10,20),0.222),
                (datetime.date(2047,2,20),0.295),
                (datetime.date(2051,3,20),0.36065),
                (datetime.date(2062,1,26),0.466),
                (datetime.date(2086,11,2),0.655),
                (datetime.date(2120,6,30),0.806103),
                ]

In [ ]:
class Bond():
    def __init__(self, acc, mat, cpn, yld, c_dt=datetime.date(2021, 8, 6)):
        '''
        acc = first interest accrual date
        mat = maturity date
        cpn = coupon in percent

        c_dt = current date
        yld = yield in percent
        '''
        self.acc = acc
        self.mat = mat
        self.cpn = cpn
        self.tenor = mat.year - acc.year
        self.cpn_dates = self.get_cpn_dates()
        self.yld = yld
        self.c_dt = c_dt

    def get_cpn_dates(self):
        '''
        Output: list of cpn payment dates using first accrual, assume Act/Act Annual
        '''
        return [datetime.date(self.acc.year + i + 1, self.acc.month, self.acc.day) for i in range(self.tenor)]

    def get_discount_factors(self):
        '''
        Output: list of discount factors from cpn_dates, assuming flat yield
        '''
        val_cpn_dates = [i for i in self.cpn_dates if i > self.c_dt]
        year_frac = [(i - self.c_dt).days / 365. for i in val_cpn_dates]
        discount_rates = [self.yld / 100.] * len(val_cpn_dates)
        return [1 / ((1 + discount_rates[i]) ** year_frac[i]) for i in range(len(year_frac))]

    def dirty_px(self):
        '''
        Inputs: c_dt, maturity, cpn, yld
        Output: dirty price
        '''
        disc_fact = self.get_discount_factors()
        return sum([d * self.cpn for d in disc_fact]) + (100 * disc_fact[-1])

    def clean_px(self):
        '''
        Inputs: c_dt, maturity, cpn, yld
        Output: dirty price
        '''
        next_year_frac = ([i for i in self.cpn_dates if i > self.c_dt][0] - self.c_dt).days / 365.
        next_disc_fact = self.get_discount_factors()[0]
        dirty_px = self.dirty_px()
        return dirty_px - (1 - next_year_frac) * next_disc_fact * self.cpn

    def dv01(self):
        '''
        Inputs: c_dt, maturity, cpn, yld
        Output: dv01

        val_cpn_dates = [i for i in self.cpn_dates if i > self.c_dt]
        w = 1-(val_cpn_dates[0] - self.c_dt).days/365.
        y = self.yld/100.
        C = self.cpn/100.
        N = float(len(val_cpn_dates))
        P = self.dirty_px()/100.
        return -1*(((1+y)**w)*((C/(y**2))*((1/((1+y)**N))-1) + N*((C/y) -1)*(1/((1+y)**(N+1))))+(w/(1+y))*P)
        '''
        px_base = self.dirty_px()
        start_yld = self.yld
        self.yld = start_yld + 0.01
        px_up = self.dirty_px()
        self.yld = start_yld - 0.01
        px_down = self.dirty_px()
        self.yld = start_yld
        return 100 * (px_down - px_up) / 2.0

    def convexity(self):
        '''
        Inputs: c_dt, maturity, cpn, yld
        Output: convexity
        val_cpn_dates = [i for i in self.cpn_dates if i > self.c_dt]
        w = 1 - (val_cpn_dates[0] - self.c_dt).days / 365.
        y = self.yld / 100.
        C = self.cpn / 100.
        N = float(len(val_cpn_dates))
        P = self.dirty_px()
        t1 = (-w / ((1 + y) ** 2)) * P + (w / (1 + y)) * dv01
        t2 = (N * (N + 1 - w)) * ((1 - (C / y)) / ((1 + y) ** (N + 2 - w)))
        t3 = (2 * C * N / (y ** 2)) / ((1 + y) ** (N + 1 - w))
        t4 = C * (((1 + y) ** w) / (y ** 3)) * (1 - 1 / ((1 + y) ** N)) * (2 - ((w * y) / (1 + y)))
        return (t1 + t2 - t3 + t4)/100
        '''
        dv01_base = self.dv01()
        yld_base = self.yld
        self.yld = yld_base + 0.01
        dv01_up = self.dv01()
        self.yld = yld_base - 0.01
        dv01_down = self.dv01()
        self.yld = yld_base
        return 100 * abs(dv01_up - dv01_down) / 2.

    def mod_duration(self):
        '''
        Inputs: c_dt, maturity, cpn, yld
        Output: modified duration
        '''
        dv01 = self.dv01()
        dirty_px = self.dirty_px()
        return 100 * (1 / dirty_px) * dv01

In [ ]:
def yld_scenarios(bd,yld_chg):
    '''
    bd: A bond object as defined above
    yld_chg: basis points to move yield up and down
    return: list of dirty pxs,dv01,dollar convexity
    '''
    start_yld = bd.yld
    yld_range = range(-yld_chg,yld_chg+1)
    prices = []
    dv01s = []
    convs = []
    for y in yld_range:
        bd.yld = start_yld+y/100
        prices.append(bd.dirty_px())
        dv01s.append(bd.dv01())
        convs.append(bd.convexity())
    return prices,dv01s,convs

def bond_scenarios(bd,t_chg,yld_chg,roll=False):
    '''
    :param bd: bond object
    :param t_chg: how many years forward
    :param y_ch: how many bp shocks
    :param roll: roll down the yield curve
    :return: 3 dataframes of prices,dv01s,convexities index by year, columns are yield shocks
    '''
    cols = range(-yld_chg,yld_chg+1)
    px_df = pd.DataFrame(columns=cols)
    dv_df = pd.DataFrame(columns=cols)
    conv_df = pd.DataFrame(columns=cols)
    start_yr = bd.c_dt.year
    start_yld = bd.yld
    for i in range(t_chg+1):
        bd.yld = start_yld
        eval_date = datetime.date(start_yr+i,bd.c_dt.month,bd.c_dt.day)
        if roll:
            xp = [datetime.date(j[0].year+i,j[0].month,j[0].day).toordinal() for j in aust_yld_crv]
            fp = [k[1] for k in aust_yld_crv]
            bd.yld = np.interp(bd.mat.toordinal(),xp,fp)
        bd.c_dt = datetime.date(start_yr+i,bd.c_dt.month,bd.c_dt.day)
        prices, dv01s, convs = yld_scenarios(bd,yld_chg)
        px_df.loc[eval_date] = prices
        dv_df.loc[eval_date] = dv01s
        conv_df.loc[eval_date] = convs
    return px_df,dv_df,conv_df

def flattener_scenarios(bd1,bd2):
    '''

    :param bd1: px,dv01,conv for bond1 - short this bond
    :param bd2: px,dv01,conv for bond2 - long this bond
    :return:
    '''

    scalar_1 = -100/bd1[1][0][0]
    scalar_2 = 100/bd2[1][0][0]

    px_df_30y, dv_df_30y, conv_df_30y = bd1[0]*scalar_1,bd1[1]*scalar_1,bd1[2]*scalar_1/100
    px_df_100y, dv_df_100y, conv_df_100y = bd2[0]*scalar_2,bd2[1]*scalar_2,bd2[2]*scalar_2/100
    pnl_30y = 100*(px_df_30y - px_df_30y[0][0])
    pnl_100y = 100*(px_df_100y - px_df_100y[0][0])
    total_pnl = pnl_30y+pnl_100y
    total_dv = dv_df_30y+dv_df_100y
    total_conv = conv_df_30y+conv_df_100y
    return  total_pnl,total_dv,total_conv

def x_intercept(total_df):
    '''
    find the x-intercept of incremental scenarios
    '''
    cols = list(total_df.index)
    input = 0
    intercepts = []
    for c in cols:
        df = total_df.T[c]
        df.iloc[(df-input).abs().argsort()[:2]]
        intercepts.append((c,sum(df.iloc[(df-input).abs().argsort()[:2]].index)/2.))
    return intercepts

In [ ]:
bond1 = Bond(datetime.date(2020,3,20),datetime.date(2051,3,20),0.75, 0.369605)
bond2 = Bond(datetime.date(2020,6,30),datetime.date(2120,6,30),0.85, 0.806103)

In [ ]:
bond_sc1 = bond_scenarios(bond1,5,400)
bond_sc2 = bond_scenarios(bond2,5,400)

In [ ]:
total_pnl,total_dv,total_conv = flattener_scenarios(bond_sc1,bond_sc2)

In [ ]:
x_label = 'Yield Shift (bps)'
y_label = "PnL (€k)"

In [ ]:
f = plt.figure()
total_pnl[range(-150,151)].loc[datetime.date(2021,8,6)].T.plot()
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title('Instantaneous PnL Scenario')
f.savefig('pnl_base.png',dpi=120)
plt.show()

In [ ]:
f2 = plt.figure()
(0.5*total_conv[range(-200,200)].loc[datetime.date(2021,8,6)].T).plot()
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title('Incremental Convexity PnL')
f2.savefig('conv_base.png',dpi=120)
plt.show()

In [ ]:
total_dv[range(-5,6)]

In [ ]:
bond_sc2[2].T.plot()

In [ ]:
bond3 = Bond(datetime.date(2020,3,20),datetime.date(2051,3,20),0.75, 0.369605)
bond4 = Bond(datetime.date(2020,6,30),datetime.date(2120,6,30),0.85, 0.806103)

In [ ]:
bond_sc3 = bond_scenarios(bond3,5,400,True)
bond_sc4 = bond_scenarios(bond4,5,400,True)

In [ ]:
total_pnl_2,total_dv_2,total_conv_2 = flattener_scenarios(bond_sc3,bond_sc4)

In [ ]:
x_intercept(total_conv)

In [ ]:
x_intercept(total_conv_2)

In [ ]:
comp_conv = pd.DataFrame(columns=['Flat','Roll'])
comp_conv['Flat'] = total_conv[0]
comp_conv['Roll'] = total_conv_2[0]
comp_conv